In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow.keras as keras

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.models import load_model #save and load models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import average_precision_score
import tensorflow.keras.backend as k

2025-01-22 11:11:59.512435: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 11:11:59.512882: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-22 11:11:59.515034: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-22 11:11:59.520814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737555119.530837 2407363 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737555119.53

In [2]:
df = pd.read_csv('data/adult.data',  names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [3]:
df1 = pd.read_csv("data/adult.test",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',\
        na_values="?")

In [4]:
data = pd.concat([df, df1], ignore_index= True)

In [5]:
names_cloud = data.columns.tolist()
print(names_cloud)
X = np.array(data[names_cloud])
print(X.shape)

['Age', 'Workclass', 'fnlwgt', 'Education', 'Education-Num', 'Martial Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Country', 'Target']
(48842, 15)


In [6]:
y = (data['Target'].map({"<=50K":0,">50K":1})).values
print(pd.value_counts(pd.Series(y)))
data.drop('Target',axis=1, inplace =True,)

0    37155
1    11687
Name: count, dtype: int64


/tmp/ipykernel_2407363/3481380141.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(pd.Series(y)))


In [7]:
categorical_features = data.select_dtypes(include=['object']).columns
print(categorical_features)
ohc_category = ['Workclass', 'Education', 'Martial Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
df_ohc = pd.get_dummies(data, columns = ohc_category)
print(df_ohc.shape)
df_ohc.head()

Index(['Workclass', 'Education', 'Martial Status', 'Occupation',
       'Relationship', 'Race', 'Sex', 'Country'],
      dtype='object')
(48842, 105)


,Age,fnlwgt,Education-Num,Capital Gain,Capital Loss,Hours per week,Workclass_Federal-gov,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,...,Country_Portugal,Country_Puerto-Rico,Country_Scotland,Country_South,Country_Taiwan,Country_Thailand,Country_Trinadad&Tobago,Country_United-States,Country_Vietnam,Country_Yugoslavia
0,39,77516,13,2174,0,40,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,9,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,7,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [8]:
names_x = df_ohc.columns.tolist()
print("Target Variable: Target")
print("Predictors: "+str(names_x))
x = np.array(df_ohc[names_x])
print("Number of data samples : {0:d}".format(x.shape[0]))
print("Number of Predictor Features : {0:d}".format(x.shape[1]))

Target Variable: Target
Predictors: ['Age', 'fnlwgt', 'Education-Num', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Workclass_Federal-gov', 'Workclass_Local-gov', 'Workclass_Never-worked', 'Workclass_Private', 'Workclass_Self-emp-inc', 'Workclass_Self-emp-not-inc', 'Workclass_State-gov', 'Workclass_Without-pay', 'Education_10th', 'Education_11th', 'Education_12th', 'Education_1st-4th', 'Education_5th-6th', 'Education_7th-8th', 'Education_9th', 'Education_Assoc-acdm', 'Education_Assoc-voc', 'Education_Bachelors', 'Education_Doctorate', 'Education_HS-grad', 'Education_Masters', 'Education_Preschool', 'Education_Prof-school', 'Education_Some-college', 'Martial Status_Divorced', 'Martial Status_Married-AF-spouse', 'Martial Status_Married-civ-spouse', 'Martial Status_Married-spouse-absent', 'Martial Status_Never-married', 'Martial Status_Separated', 'Martial Status_Widowed', 'Occupation_Adm-clerical', 'Occupation_Armed-Forces', 'Occupation_Craft-repair', 'Occupation_Exec-managerial', 

## MOdelo Alvo ORiginal

In [9]:
x = x.astype('float32')
scaler = MinMaxScaler()

x = scaler.fit_transform(x)


data_size = 10000
ns = 5 #number of shadow models for one data_size

nout = 1
seed = 7
np.random.seed(seed)
sh = np.arange(x.shape[0])
np.random.shuffle(sh)
target_rep = np.zeros((1,x.shape[0]))
target_rep[0,:] = sh
print(sh)

[23509 38928 23796 ...   919 38467 10742]


In [10]:
k.clear_session()

xtr_target = x[sh[:data_size]]
ytr_target = y[sh[:data_size]]
xts_target = x[sh[data_size:data_size*2]]
yts_target = y[sh[data_size:2*data_size]]

shadow_rep = np.zeros((5,x.shape[0]-2*data_size))
sh1 = sh[2*data_size:]
xtr_att = np.zeros((2*data_size*ns,1))
ytr_att = np.zeros((2*data_size*ns,1))
xtr_att_truelabels = np.zeros((2*data_size*ns,))

model_target = Sequential()
model_target.add(Dense(5, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
model_target.add(Dense(1, activation='sigmoid', name = 'output'))
opt = keras.optimizers.Adam(learning_rate=0.001, decay = 1e-7)
model_target.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

print(model_target.summary())

hist_target = model_target.fit(xtr_target, ytr_target,
                  batch_size = 32,
                  epochs = 100,
                  validation_data=(xts_target, yts_target), shuffle=True, verbose=0)

print('\n\nFor target model with training datasize = %d'%data_size)
print('Training accuracy = %f'%hist_target.history['accuracy'][-1])
print('Validation accuracy = %f'%hist_target.history['val_accuracy'][-1])

model_target_name = 'UCI_Adult_target_'+str(data_size)+'.h5'

model_target.save(model_target_name)

ytemp_tr_target = model_target.predict(xtr_target)
ytemp_ts_target = model_target.predict(xts_target)

xts_att = np.vstack((ytemp_tr_target,ytemp_ts_target))
yts_att = np.zeros((2*data_size,1))
yts_att[data_size:2*data_size] = 1
xts_att_truelabels = np.vstack((ytr_target,yts_target))
xts_att_dict = {'xts_att':xts_att,'yts_att':yts_att,'xts_att_truelabels':xts_att_truelabels}

fname = './att_test_data_'+str(data_size)
np.save(fname,xts_att_dict)
datafile = './data_adult_target_'+str(data_size)
np.save(datafile,target_rep)

/media/emanuel/sdd_sata/projetos/projeto_mestrado_ofc/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1737555124.333375 2407363 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/emanuel/sdd_sata/projetos/projeto_mestrado_ofc/.venv/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │           530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536 (2.09 KB)

 Trainable params: 536 (2.09 KB)

 Non-trainable params: 0 (0.00 B)

None




For target model with training datasize = 10000
Training accuracy = 0.850500
Validation accuracy = 0.849500
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 300us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 300us/step


## Modelo Sombra Original

In [11]:
for i in np.arange(ns):

    np.random.shuffle(sh1)
    shadow_rep[i,:] = sh1
    
    xtr_shadow = x[sh1[:data_size]]
    ytr_shadow = y[sh1[:data_size]]
    xts_shadow = x[sh1[data_size:2*data_size]]
    yts_shadow = y[sh1[data_size:2*data_size]]

    model_shadow = Sequential()
    model_shadow.add(Dense(5, input_shape =(x.shape[1],), activation='sigmoid', name = 'hidden'))
    model_shadow.add(Dense(1, activation='sigmoid', name = 'output'))
    opt = keras.optimizers.Adam(learning_rate=0.001, decay=1e-7)
    model_shadow.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    if i == 0:
        print("Shadow Model Summary")
        print(model_shadow.summary())
        
    hist_shadow = model_shadow.fit(xtr_shadow, ytr_shadow,
                  batch_size = 32,
                  epochs = 100,
                  validation_data=(xts_shadow, yts_shadow), shuffle=True, verbose=0)
    
    print("Shadow model no: %d"%i)
    print('\n\nFor shadow model with training datasize = %d'%data_size)
    print('Training accuracy = %f'%hist_shadow.history['accuracy'][-1])
    print('Validation accuracy = %f'%hist_shadow.history['val_accuracy'][-1])

    ytemp11 = model_shadow.predict(xtr_shadow)
    ytemp22 = model_shadow.predict(xts_shadow)

    model_shadow_name = 'UCI_Adult_shadow_'+str(data_size)+'_'+str(i)+'.h5'
    print(model_shadow_name)
    model_shadow.save(model_shadow_name)

    xtr_att[i*2*data_size:(i+1)*2*data_size] = np.vstack((ytemp11,ytemp22))
    ytr_att[((i*2)+1)*data_size:(i+1)*2*data_size] = 1
    xtr_att_truelabels[i*2*data_size:(i+1)*2*data_size] = np.hstack((ytr_shadow,yts_shadow))

datafile = './data_adult_shadow_'+str(data_size)
np.save(datafile,shadow_rep)
xtr_att_dict = {'xtr_att':xtr_att,'ytr_att':ytr_att,'xtr_att_truelabels':xtr_att_truelabels}
fname = './att_train_data_'+str(data_size)
np.save(fname,xtr_att_dict)

Shadow Model Summary


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │           530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536 (2.09 KB)

 Trainable params: 536 (2.09 KB)

...

UCI_Adult_shadow_10000_0.h5
Shadow model no: 1


For shadow model with training datasize = 10000
Training accuracy = 0.859800
Validation accuracy = 0.849400
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step


UCI_Adult_shadow_10000_1.h5
Shadow model no: 2


For shadow model with training datasize = 10000
Training accuracy = 0.859800
Validation accuracy = 0.848100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step


UCI_Adult_shadow_10000_2.h5
Shadow model no: 3


For shadow model with training datasize = 10000
Training accuracy = 0.862600
Validation accuracy = 0.844500
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step


UCI_Adult_shadow_10000_3.h5
Shadow model no: 4


For shadow model with training datasize = 10000
Training accuracy = 0.857000
Validation accuracy = 0.849900
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 273us/step


UCI_Adult_shadow_10000_4.h5


## Modelo de Ataque

In [12]:
model_attack = Sequential()
model_attack.add(Dense(5, input_shape = (xtr_att.shape[1],), activation='sigmoid', name = 'hidden'))
model_attack.add(Dense(1, activation='sigmoid', name = 'output'))
opt = keras.optimizers.Adam(learning_rate=0.001, decay=1e-7)

model_attack.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

print("Attack Model Summary")
print(model_attack.summary())

hist_attack = model_attack.fit(xtr_att, ytr_att,
                  batch_size = 32,
                  epochs = 100,
                  validation_data=(xts_att, yts_att), shuffle=True, verbose=0)

print('\n\nFor attack model with training datasize = %d'%xtr_att.shape[0])
print('Training accuracy = %f'%hist_attack.history['accuracy'][-1])
print('Validation accuracy = %f'%hist_attack.history['val_accuracy'][-1])

y_score = model_attack.predict(xts_att)
average_precision = average_precision_score(yts_att, y_score)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Attack Model Summary


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 5)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16 (64.00 B)

 Trainable params: 16 (64.00 B)

 Non-trainable params: 0 (0.00 B)

None


For attack model with training datasize = 100000
Training accuracy = 0.501610
Validation accuracy = 0.500000
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step
Average precision-recall score: 0.50


## COmbinando amostras entre alvo e sombras no conjunto de teste do ataque

In [13]:
# # First, collect predictions from both target and shadow models for test set
# # Assuming we want to use 20% of each shadow model's predictions in the test set

# Calculate how many samples we want from each shadow model
shadow_samples_per_model = int(0.2 * data_size)  # 20% of data_size
total_shadow_samples = shadow_samples_per_model * ns  # ns is number of shadow models

# Initialize arrays for combined test set
xts_att_combined = np.zeros((2*data_size + total_shadow_samples, 1))
yts_att_combined = np.zeros((2*data_size + total_shadow_samples, 1))
xts_att_truelabels_combined = np.zeros((2*data_size + total_shadow_samples,))

# First, add all target model predictions
xts_att_combined[:2*data_size] = np.vstack((ytemp_tr_target, ytemp_ts_target))
yts_att_combined[:data_size] = 0  # members (target train)
yts_att_combined[data_size:2*data_size] = 1  # non-members (target test)
xts_att_truelabels_combined[:2*data_size] = np.hstack((ytr_target, yts_target))

# Then add portions of shadow model predictions
current_index = 2*data_size

for i in np.arange(ns):
    # Load or get predictions from each shadow model
    model_shadow = load_model(f'UCI_Adult_shadow_{data_size}_{i}.h5')

    # Get predictions from shadow model
    shadow_tr_preds = model_shadow.predict(xtr_shadow)  # predictions on shadow train data
    shadow_ts_preds = model_shadow.predict(xts_shadow)  # predictions on shadow test data

    # Randomly select samples from both train and test predictions
    combined_preds = np.vstack((shadow_tr_preds, shadow_ts_preds))
    combined_labels = np.hstack((np.zeros(data_size), np.ones(data_size)))  # 0 for members, 1 for non-members
    combined_true_labels = np.hstack((ytr_shadow, yts_shadow))

    # Randomly select indices
    random_indices = np.random.choice(2*data_size, shadow_samples_per_model, replace=False)

    # Add selected predictions and their labels to test set
    end_index = current_index + shadow_samples_per_model
    xts_att_combined[current_index:end_index] = combined_preds[random_indices].reshape(-1, 1)
    yts_att_combined[current_index:end_index] = combined_labels[random_indices].reshape(-1, 1)
    xts_att_truelabels_combined[current_index:end_index] = combined_true_labels[random_indices]

    current_index = end_index

# Save the combined test data
xts_att_dict = {
    'xts_att': xts_att_combined,
    'yts_att': yts_att_combined,
    'xts_att_truelabels': xts_att_truelabels_combined
}
fname = f'./att_test_data_combined_{data_size}'
np.save(fname, xts_att_dict)


hist_attack = model_attack.fit(xtr_att, ytr_att,
                             batch_size=32,
                             epochs=100,
                             validation_data=(xts_att_combined, yts_att_combined),
                             shuffle=True, verbose=0)


print('\n\nFor attack model with training datasize = %d' % xtr_att.shape[0])
print('Training accuracy = %f' % hist_attack.history['accuracy'][-1])
print('Validation accuracy = %f' % hist_attack.history['val_accuracy'][-1])
y_score = model_attack.predict(xts_att_combined)
average_precision = average_precision_score(yts_att_combined, y_score)
print('Average precision-recall score: {0:0.2f}'.format(average_precision))

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 281us/step


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 280us/step


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 273us/step


For attack model with training datasize = 100000
Training accuracy = 0.499660
Validation accuracy = 0.499733
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 271us/step
Average precision-recall score: 0.50


In [14]:
for key, value in xts_att_dict.items():
  print(f'{key}: {len(value)}')

xts_att: 30000
yts_att: 30000
xts_att_truelabels: 30000
